In [1]:
from config import DATABASE_URI
import psycopg2
from PIL import Image
import io
import numpy as np

In [7]:
def get_images_from_db():
# Connect to the database
    conn = psycopg2.connect(DATABASE_URI)

    cursor = conn.cursor()
    
    # Query to get banana and apple images
    cursor.execute("SELECT fruit_name, image FROM fruits")
    rows = cursor.fetchall()
    
    images = []
    labels = []
    for row in rows:
        fruit_name, binary_data = row
        image = Image.open(io.BytesIO(binary_data))
        image = image.resize((128, 128))  # Resize for consistency
        images.append(np.array(image))
        labels.append(fruit_name)
    
    cursor.close()
    conn.close()
    
    return np.array(images), np.array(labels)

images, labels = get_images_from_db()
print(f'Retrieved {len(images)} images from the database.')

Retrieved 43845 images from the database.


In [8]:
# Normalize the images
images = images / 255.0

# Encode labels as integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [12]:

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

MemoryError: Unable to allocate 12.8 GiB for an array with shape (35076, 128, 128, 3) and data type float64

In [10]:
# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\Javier Alarcon\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

In [11]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

MemoryError: Unable to allocate 6.42 GiB for an array with shape (35076, 128, 128, 3) and data type float32

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

47/47 - 1s - 30ms/step - accuracy: 0.9693 - loss: 0.2248
Test accuracy: 0.969292402267456


In [9]:
# Predict on test data
predictions = model.predict(X_test)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


In [10]:
# Print some of the predictions to see how the model performs
for i in range(10):
    print(f'Actual: {label_encoder.inverse_transform([y_test[i]])[0]}, Predicted: {predicted_labels[i]}')

Actual: Banana, Predicted: Banana
Actual: Banana, Predicted: Apple
Actual: Apple, Predicted: Apple
Actual: Banana, Predicted: Banana
Actual: Apple, Predicted: Apple
Actual: Apple, Predicted: Apple
Actual: Banana, Predicted: Banana
Actual: Banana, Predicted: Banana
Actual: Apple, Predicted: Apple
Actual: Banana, Predicted: Banana
